# TF-IDF 기반 피처 벡터 생성 - 네이버 영화 리뷰
* TF-IDF는 토큰화 이전의 문장 형태의 데이터를 사용

### Cleaning된 문장 데이터를 로딩하여 사용

In [2]:
import pandas as pd
train_data = pd.read_csv('DataSet_NaverMovieReview/train_data_prep_sent.csv')
test_data = pd.read_csv('DataSet_NaverMovieReview/test_data_prep_sent.csv')
print('훈련 샘플의 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트 샘플의 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

훈련 샘플의 개수 : 145393
테스트 샘플의 개수 : 48852


In [3]:
train_data.head()

,index,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


# TF-IDF 기반 피처 벡터 생성 
* sklearn 라이브러리의 TfidfVectorizer로 학습하여 Feature Vectore 생성 모델 구축
   - from sklearn.feature_extraction.text import TfidfVectorizer
   - 모델 생성 : *model* = **TfidfVectorizer**(tokenizer=*tokenizer_func*, *optional parameters*)
   - 모델 학습 : *model*.**fit**(*train_text_data*)
   - Feature Vertor로 변환 : *feature_vector* = *model*.**transform**(*text_data*)
* 실행시간 10분 정도 걸림

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [4]:
# Tokenizer 정의
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text, stem=True)
    #tokens = okt.morphs(text)
    return tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_model = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_model.fit(train_data['document'])
train_tfidf = tfidf_model.transform(train_data['document'])

In [6]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(stop_words=stopwords, tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

In [7]:
tfidf_model.fit(train_data['document'])

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                stop_words=['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과',
                            '도', '를', '으로', '자', '에', '와', '한', '하다'],
                tokenizer=<function okt_tokenizer at 0x000001F300EB0D60>)

In [10]:
train_tfidf = tfidf_model.transform(train_data['document'])

In [11]:
train_tfidf

<145393x93857 sparse matrix of type '<class 'numpy.float64'>'
	with 2300108 stored elements in Compressed Sparse Row format>

In [15]:
X_test_tfidf = tfidf_model.transform(test_data['document'])

### Tfidf 모델과 데이터의 feature vector 저장

In [9]:
import joblib
joblib.dump(tfidf_model, 'DataSet_NaverMovieReview/tfidf_model.pkl')

['DataSet_NaverMovieReview/tfidf_model.pkl']

In [12]:
from scipy.sparse import save_npz, load_npz
save_npz('DataSet_NaverMovieReview/X_train_Tfidf.npz', train_tfidf)

In [ ]:
save_npz('DataSet_NaverMovieReview/X_test_Tfidf.npz', test_tfidf)

In [13]:
train_tfidf_loaded = load_npz('DataSet_NaverMovieReview/X_train_Tfidf.npz')

In [14]:
(train_tfidf != train_tfidf_loaded).nnz

0